In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# TODO: See gpt for Skin Tone Detection...

Load image and convert to different colour spaces

In [ ]:
# Convert from BGR to YCrCb
img_bgr = cv2.imread("./outputs/spill01_seg_img.png")
img_ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
y, cr, cb = cv2.split(img_ycrcb)
# Skin detection mask based on Cr and Cb
skin_mask = cv2.inRange(img_ycrcb, (0, 133, 77), (255, 173, 127))
# Detect where green is significantly stronger than red and blue
b, g, r = cv2.split(img_bgr)
green_spill = (g > r + 30) & (g > b + 30)
# Reduce Cb
cb_corrected = cb.copy()
cb_corrected[green_spill] = np.clip(cb_corrected[green_spill] - 10, 0, 255)
# Recompute skin mask with corrected Cb
corrected_skin_mask = ((cr >= 133) & (cr <= 173)) & ((cb_corrected >= 77) & (cb_corrected <= 127))
corrected_skin_mask = corrected_skin_mask.astype(np.uint8) * 255
# Clean up mask with  morphological ops
the_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
cleaned_mask = cv2.morphologyEx(corrected_skin_mask, cv2.MORPH_CLOSE, the_kernel, iterations=2)
cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_CLOSE, the_kernel, iterations=2)


In [ ]:
plt.imshow(cleaned_mask)
